In [1]:
import sys
sys.path.append('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/')
import pandas as pd 
import numpy as np 
import datasets
from datasets import Dataset as TFDataset 
import pickle
from bm25.rank_bm25 import BM25Okapi
import random
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
from transformers import PreTrainedTokenizer, AutoTokenizer
import multiprocessing

In [2]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def tokenize_function(examples, corpus_col_name, tokenizer, max_length):
    if corpus_col_name in examples:
        return tokenizer(examples[corpus_col_name], 
            add_special_tokens=False, # don't add special tokens when preprocess
            truncation=True, 
            max_length=max_length,
            return_attention_mask=False,
            return_token_type_ids=False)

In [4]:
def construct_query_list_from_sessions(sessions_df:pd.DataFrame, product_map:dict, max_seq_len:int, product_corpus:list):
    query_list = []
    for sess in tqdm(sessions_df.itertuples(), total=sessions_df.shape[0]):
        sess_locale = sess.locale
        prev_items = eval(sess.prev_items.replace(' ', ','))[-max_seq_len : ]
        prev_items = [product_map.get(sess_locale+'_'+item, 0) for item in prev_items]
        sess_query = sum([product_corpus[item] for item in prev_items], [])
        query_list.append(sess_query)
    return query_list

# Merge valid BM25 score

In [5]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'

In [6]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)


In [7]:
merged_candidates_feature = read_merged_candidates_feature()
product_data = read_product_data()
valid_sessions = read_valid_sessions()

In [8]:
TOKENIZER_NAME = 'xlm-roberta-base'
DESC_MAX_LENGTH = 500

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
        TOKENIZER_NAME,
        use_fast=False,
)

In [10]:
desc_corpus = product_data[['desc', 'brand', 'color', 'size', 'model', 'material', 'author']]
padding_df = pd.DataFrame({'desc' : [''], 'brand' : [''], 'color' : [''], 'size' : [''], 'model' : [''], 'material' : [''], 'author' : ['']})
desc_corpus = pd.concat([padding_df, desc_corpus]).reset_index(drop=True) # add padding product
desc_corpus['desc'] = desc_corpus['desc'].fillna('')
desc_corpus['brand'] = desc_corpus['brand'].fillna('')
desc_corpus['color'] = desc_corpus['color'].fillna('')
desc_corpus['size'] = desc_corpus['size'].fillna('')
desc_corpus['model'] = desc_corpus['model'].fillna('')
desc_corpus['material'] = desc_corpus['material'].fillna('')
desc_corpus['author'] = desc_corpus['author'].fillna('')

desc_corpus['desc'] = desc_corpus['desc'] + ' ' + desc_corpus['brand'] + ' ' + desc_corpus['color'] + ' ' + desc_corpus['size'] + ' ' + desc_corpus['model'] \
    + ' ' + desc_corpus['material'] + ' ' + desc_corpus['author']
desc_corpus['desc'] = desc_corpus['desc'].apply(lambda x : x.lower())

In [11]:
desc_corpus = TFDataset.from_pandas(desc_corpus, preserve_index=False)
desc_corpus = desc_corpus.map(partial(tokenize_function, corpus_col_name='desc', tokenizer=tokenizer, max_length=DESC_MAX_LENGTH), 
                                num_proc=8, remove_columns=['desc'], batched=True)
desc_corpus_list = desc_corpus['input_ids']

Parameter 'function'=<function Dataset.map.<locals>.decorate.<locals>.decorated at 0x7f4e14bc04c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [12]:
with open('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/bm25/cache/desc_bm25.pkl', 'rb') as f:
    desc_BM25 = pickle.load(f)

In [13]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]
merged_candidates

,sess_id,sess_locale,product
0,0,DE,355165591X
1,0,DE,3833237058
2,0,DE,B00CIXSI6U
3,0,DE,B00NVDOWUW
4,0,DE,B00NVDP3ZU
...,...,...,...
78842194,261815,UK,B0BCX524Y6
78842195,261815,UK,B0BCX6QB4L
78842196,261815,UK,B0BFPJYXQL
78842197,261815,UK,B0BH3X67S3


In [14]:
product_index = product_data[['id', 'locale']]
product_index['product_index'] = product_index.index + 1

/tmp/ipykernel_419093/3969984160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_index['product_index'] = product_index.index + 1


In [15]:
product_index.query("id=='B0079JI4DU'")

,id,locale,product_index
101952,B0079JI4DU,DE,101953
1516940,B0079JI4DU,IT,1516941


In [16]:
merged_candidates = merged_candidates.merge(product_index, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'id'])
merged_candidates['product_index'] = merged_candidates['product_index'].fillna(0)
merged_candidates['product_index'] = merged_candidates['product_index'].astype('int64')
merged_candidates.drop(columns=['id', 'locale'], inplace=True)
assert len(merged_candidates) == len(merged_candidates_feature)
merged_candidates_grouped = merged_candidates.groupby(by='sess_id')['product_index'].apply(list)

In [17]:
locale_product_map = {}
for row in tqdm(product_data.itertuples(), total=product_data.shape[0]):
    locale_product_map[row.locale+'_'+row.id] = row.Index + 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1551057/1551057 [00:07<00:00, 210886.78it/s]


In [18]:
# construct query list 
valid_query_list = construct_query_list_from_sessions(valid_sessions, locale_product_map, 5, product_corpus=desc_corpus_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 261816/261816 [00:17<00:00, 15015.10it/s]


In [19]:
def get_sess_scores(sess):
    sess_id = sess['sess_id']
    scores = desc_BM25.get_batch_scores(valid_query_list[sess_id], merged_candidates_grouped[sess_id])
    return {'sess_bm25_scores' : scores}

In [20]:
# about 23 mins
datasets.set_progress_bar_enabled(False)
valid_query_dataset = TFDataset.from_dict({'sess_id' : list(range(len(valid_query_list)))})
valid_query_dataset = valid_query_dataset.map(get_sess_scores, num_proc=20, batched=False)
datasets.set_progress_bar_enabled(True)

In [21]:
valid_scores_list = valid_query_dataset['sess_bm25_scores']

In [22]:
merged_bm25_scores = []
for scores_set in tqdm(valid_scores_list):
    for s in scores_set:
        merged_bm25_scores.append(s)
assert len(merged_bm25_scores) == len(merged_candidates)
assert len(merged_bm25_scores) == len(merged_candidates_feature)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 261816/261816 [00:11<00:00, 22740.32it/s]


In [23]:
merged_candidates_feature['desc_BM25_scores'] = merged_bm25_scores

In [24]:
cast_dtype(merged_candidates_feature)
merged_candidates_feature.to_parquet(merged_candidates_feature_path)

In [25]:
merged_candidates_feature.query('sess_id==200002').sort_values(by=['title_BM25_scores'], ascending=False)[['sess_id', 'sess_locale', 'product', 'desc_BM25_scores', 'title_BM25_scores', 'normalized_sasrec_scores_2']][:50]

,sess_id,sess_locale,product,desc_BM25_scores,title_BM25_scores,normalized_sasrec_scores_2
60269002,200002,UK,B07N7J47T8,803.123779,543.995178,1.000310e-01
60269003,200002,UK,B07N7JN882,803.123779,527.934204,2.013040e-03
60268999,200002,UK,B07N7FG1WF,803.123779,523.200317,4.660428e-01
60269000,200002,UK,B07N7G92H8,803.123779,518.097717,1.374451e-02
60269085,200002,UK,B093QDB5BJ,803.123779,512.303589,3.898879e-03
60269001,200002,UK,B07N7GD457,750.349182,510.861908,1.311428e-04
60269083,200002,UK,B093QC527W,803.123779,508.312561,1.500493e-03
60269135,200002,UK,B09P3TWTSM,750.349182,499.545288,1.750574e-03
60269084,200002,UK,B093QCCVNN,750.349182,499.199036,1.128479e-03
60269005,200002,UK,B07N7SKK2F,803.123779,498.915680,2.880406e-04


In [26]:
merged_candidates

,sess_id,sess_locale,product,product_index
0,0,DE,355165591X,127299
1,0,DE,3833237058,248194
2,0,DE,B00CIXSI6U,115708
3,0,DE,B00NVDOWUW,55370
4,0,DE,B00NVDP3ZU,100302
...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,982599
78842195,261815,UK,B0BCX6QB4L,1046447
78842196,261815,UK,B0BFPJYXQL,1187691
78842197,261815,UK,B0BH3X67S3,983386


In [27]:
merged_candidates_feature

,sess_id,sess_locale,product,target,sess_avg_price,product_price,product_freq,sasrec_scores_3,normalized_sasrec_scores_3,sasrec_scores_2,normalized_sasrec_scores_2,seqmlp_scores,normalized_seqmlp_scores,narm_scores,normalized_narm_scores,gru4rec_scores_2,normalized_gru4rec_scores_2,title_BM25_scores,desc_BM25_scores
0,0,DE,355165591X,0.0,43.256542,8.990000,51.0,2.230508,7.658405e-09,0.512931,1.377575e-09,6.044256,3.054954e-09,3.628220,1.469988e-13,5.093824,1.288155e-10,0.000000,0.455206
1,0,DE,3833237058,0.0,43.256542,22.000000,84.0,9.605231,1.221631e-05,9.325538,9.255110e-06,10.732503,3.319589e-07,15.734776,2.661486e-08,13.133082,3.993682e-07,89.696823,0.904280
2,0,DE,B00CIXSI6U,0.0,43.256542,6.470000,7.0,0.714114,1.681035e-09,-0.115904,7.345399e-10,4.902086,9.749146e-10,7.462822,6.802375e-12,4.295491,5.797709e-11,0.000000,0.000000
3,0,DE,B00NVDOWUW,0.0,43.256542,11.990000,166.0,8.750996,5.199363e-06,8.507557,4.084482e-06,13.401752,4.789881e-06,9.774860,6.866981e-11,10.362890,2.502041e-08,180.035004,252.282623
4,0,DE,B00NVDP3ZU,0.0,43.256542,22.990000,502.0,8.056712,2.596729e-06,5.898870,3.007453e-07,9.162767,6.908073e-08,10.392188,1.273116e-10,12.687778,2.558471e-07,148.722244,218.617142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,0.0,9.383333,16.990000,7.0,6.813615,1.076201e-03,7.203015,4.597607e-04,6.808200,4.075167e-04,7.742151,7.126370e-04,7.978776,9.559332e-04,95.027313,22.518597
78842195,261815,UK,B0BCX6QB4L,0.0,9.383333,10.990000,53.0,9.030836,9.881445e-03,10.123234,8.526421e-03,10.454871,1.562696e-02,10.198983,8.314902e-03,10.432762,1.112192e-02,95.027313,22.518597
78842196,261815,UK,B0BFPJYXQL,0.0,9.383333,10.560000,7.0,0.796892,2.623396e-06,1.711608,1.895152e-06,-0.960521,1.722791e-07,2.838342,5.286535e-06,2.107661,2.695469e-06,177.093521,130.335434
78842197,261815,UK,B0BH3X67S3,0.0,9.383333,6.830000,38.0,4.250781,8.296004e-05,6.447586,2.159998e-04,5.091795,7.323526e-05,6.170062,1.479513e-04,7.305784,4.876977e-04,33.781818,43.050674
